In [ ]:
# !pip3 install holidays

In [1]:
import pandas as pd
import holidays
import datetime
import calendar
import numpy as np

In [ ]:
df = pd.read_csv('/home/faye/data/vehicles_2018_cleaned_dataset')

In [ ]:
# Converting date columns to datetime
datetime_columns = ['DAYOFSERVICE', 'LASTUPDATE']
for column in datetime_columns:
    df[column] = pd.to_datetime(df[column])

In [ ]:
# Getting the Irish holidays - including bank holidays
# https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
irish_holidays_2018 = []
for date in holidays.Ireland(years=2018).items():
    irish_holidays_2018.append(str(date[0]))

In [ ]:
irish_holidays_2018

In [ ]:
df['Is_HOLIDAY'] = [1 if str(val).split()[0] in irish_holidays_2018 else 0 for val in df['DAYOFSERVICE']]
df['IS_WEEKDAY'] = [1 if int(val.weekday()) < 5 else 0 for val in df['DAYOFSERVICE']]
df['DAYOFWEEK'] = [calendar.day_name[val.weekday()] for val in df['DAYOFSERVICE']]
df['MONTHOFSERVICE'] = df['DAYOFSERVICE'].dt.strftime('%B')

In [ ]:
df.tail(10)

### Calculating how long it took to get to the next stop
As we know, each tripid represents one trip at a certain point of the day. So what we need to do is take away the second stop from the first stop. **actualtime_arr from the second stop - actualtime_dep from the first stop**

**Prequisites:**
* Have each row by specific trip and date.

**Method:**
* Loop through every date and tripid:
    * Initiate an empty list called time_between
    * If tripid and date match in a row:
        * Store PROGRNUMBER in a list then 
        * Loop through the list with PROGRNUMBERS:
            * add to time between: the current progrnumber actualtime_arr MINUS progrnumber previous actualtime_dep
    * Add that populated list to the existing rows in loop
    * Replace rows according to the index
    <br>
    (May cause memory error but whatever we'll try this method)

In [ ]:
# leavetimes = pd.read_csv('/home/faye/data/leavetimes_cleaned_2.csv')

In [ ]:
# leavetime_2 = leavetimes.to_feather('/home/faye/data/leavetimes_cleaned_2.feather')

In [2]:
leavetimes = pd.read_feather('/home/faye/data/leavetimes_cleaned_2.feather')

In [4]:
leavetimes.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,VEHICLEID,PLANNEDTIME_ARR,ACTUALTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_DEP
0,2018-01-01,5972116,12,119,2693211,48030,48012,48030,48012
1,2018-01-01,5966674,12,119,2693267,54001,54023,54001,54023
2,2018-01-01,5959105,12,119,2693263,60001,59955,60001,59955
3,2018-01-01,5966888,12,119,2693284,58801,58771,58801,58771
4,2018-01-01,5965960,12,119,2693209,56401,56309,56401,56323


In [5]:
datetime_columns = ['DAYOFSERVICE']
time_columns = ['ACTUALTIME_DEP', 'ACTUALTIME_ARR']
for column in datetime_columns:
    leavetimes[column] = pd.to_datetime(leavetimes[column])

for column in time_columns:
    leavetimes[column] = leavetimes[column].astype(int)

In [6]:
leavetimes = leavetimes.sort_values(by='PROGRNUMBER')
# grouped_leavetimes = leavetimes.groupby(by='TRIPID')

In [7]:
tripid = [list(leavetimes['TRIPID'].unique()), list(leavetimes['DAYOFSERVICE'].unique())]

In [8]:
# making new column with empty values
leavetimes['TOTALTIME'] = np.nan

In [9]:
# Looping through every date and tripid
for id_, date in zip(tripid[0], tripid[1]):
    # making an empty list for the time it took between each stop - will be measured in time
    time_between = []
    row = leavetimes.loc[(leavetimes['TRIPID']==id_) & (leavetimes['DAYOFSERVICE']==date)]
    prog_num = list(row['PROGRNUMBER'])
#     for stop in prog_num:
#         index = prog_num.index(stop)
#         if stop == 1:
#             time_between += [0]
#         else:
#             time_between += [(int(row['ACTUALTIME_ARR'].loc[row['PROGRNUMBER']==stop])) - (int(row['ACTUALTIME_DEP'].loc[row['PROGRNUMBER']==prog_num[index-1]]))]
    time_between = [1 if stop == 1 else (int(row['ACTUALTIME_ARR'].loc[row['PROGRNUMBER']==stop])) - (int(row['ACTUALTIME_DEP'].loc[row['PROGRNUMBER']==prog_num.index(stop)-1]))
                   for stop in prog_num]
    row.drop('TOTALTIME', axis = 1, inplace = True)
    row['TOTALTIME'] = time_between
    leavetimes.combine_first(row)

/home/faye/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/faye/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


MemoryError: Unable to allocate 6.97 GiB for an array with shape (8, 116949110) and data type int64

In [ ]:
v = leavetimes.loc[(leavetimes['TRIPID']==8089172) & (leavetimes['DAYOFSERVICE']=='2018-09-15 00:00:00')]
# progrnum = list(v['PROGRNUMBER'])
# time = []
# for stop in progrnum:
#     if stop == 1:
#         time += [0]
#     else:
#         print(v['ACTUALTIME_ARR'].loc[v['PROGRNUMBER']==stop])
#         acsh_arr = int(v['ACTUALTIME_ARR'].loc[v['PROGRNUMBER']==stop])
#         acsh_dep = int(v['ACTUALTIME_DEP'].loc[v['PROGRNUMBER']==stop-1])
#         time += [acsh_arr - acsh_dep]
# v.drop('TOTALTIME', axis = 1, inplace = True)
# v['TOTALTIME'] = time

In [ ]:
leavetimes.head(20)